<a href="https://colab.research.google.com/github/kattens/Protein-Interaction-with-LLMs/blob/main/REAL_Second_part_model_after_the_protbert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --upgrade transformers

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")


False
CUDA not available


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_file = '/content/drive/MyDrive/Phase3Project/Pairs_df_new(1).csv'
pairs_df = pd.read_csv(csv_file)

In [ ]:
# Truncate each specified column to a maximum length of 500 characters
columns = ['masked_sequence_A', 'masked_sequence_B', 'Sequence_A', 'Sequence_B']
for col in columns:
    pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)

# Find the longest string by length
pairs_df['Length'] = pairs_df['masked_sequence_A'].apply(len)
longest_string = pairs_df.loc[pairs_df['Length'].idxmax(), 'masked_sequence_A']
print(len(longest_string))


500


Create dataset class that handles both global sequences and local sequences for protein pairs, and potentially prepares for the inclusion of 3D structural data

#the base Model (without coordinates at this point):

  ### Modeling Interactions:
  The mdel could be trained to recognize which amino acids interact by learning representations of local sequences that highlight these interactions. During training, the MLM objective helps the model learn contextual embeddings that are rich in information about which amino acids tend to be near each other and under what structural contexts they interact.

  ### Attention Mechanism:
   The custom attention mechanism can be used to weigh the importance of different amino acids in the global context when predicting the masked amino acids in the local sequences. This allows your model to focus more on the parts of the global sequences that are relevant to the interactions highlighted by the local sequences.

  ### Utilizing Global Sequences:
  While the local sequences are your primary interest, the global sequences provide the context necessary for your model to understand the broader environment in which the interactions occur. Even during prediction, you should feed the model the global sequences to utilize the learned context.

  #### This modular design not only meets your current requirements but also provides a scalable framework to incorporate additional dimensions of protein sequence data analysis in the future


In [ ]:
!ls /content/drive/MyDrive/Checkpoints

added_tokens.json  generation_config.json  special_tokens_map.json  tokenizer.json
config.json	   model.safetensors	   tokenizer_config.json    vocab.txt


In [ ]:
#in this block of code we will intialize our core model for the architecture
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# Define the checkpoint directory and the specific checkpoint file
checkpoint_path = '/content/drive/MyDrive/Checkpoints/final_checkpoint.pth'

# Load the tokenizer first
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Checkpoints')

# Initialize the model from the pre-trained configuration in the Checkpoints directory
model = AutoModelForMaskedLM.from_pretrained('/content/drive/MyDrive/Checkpoints')
model.resize_token_embeddings(len(tokenizer))  # Important if you've added tokens

# Determine the device to use (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)  # Ensure checkpoint is loaded to the correct device

# Ensure all keys in the checkpoint can be loaded to the model
missing_keys, unexpected_keys = model.load_state_dict(checkpoint['model_state_dict'], strict=False)
if missing_keys or unexpected_keys:
    print(f"Missing keys in state dict: {missing_keys}")
    print(f"Unexpected keys in state dict: {unexpected_keys}")
else:
    print("Model state loaded successfully.")

# If you need the optimizer and scheduler states
optimizer = Adam(model.parameters(), lr=checkpoint.get('learning_rate', 0.001))  # Fallback to default if not in checkpoint
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
if 'scheduler_state_dict' in checkpoint:
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
else:
    print("No scheduler state found in checkpoint; using default settings.")

print("Model, tokenizer, optimizer, and scheduler loaded from checkpoint.")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Checkpoints/final_checkpoint.pth'

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

class ProteinInteractionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512, mask_probability=0.15, modes=None):
        """
        Initializes the dataset.
        Args:
            dataframe (pandas.DataFrame): The dataframe containing protein sequences.
            tokenizer (transformers.BertTokenizer): The tokenizer for encoding sequences.
            max_length (int): Maximum sequence length.
            mask_probability (float): The probability of masking a token for the masked language model.
            modes (list of str): List of modes to prepare data.
        """
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_probability = mask_probability
        self.modes = modes if modes else ['global_masked']

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        data = {}

        for mode in self.modes:
            if mode == 'global_masked':
                sequence = f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B']} [ENTITY2] [SEP]"
                input_ids, attention_mask, labels = self.random_mask_sequence(sequence)
                key_suffix = 'global_masked'
            elif mode == 'local':
                sequence = f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B']} [ENTITY2] [SEP]"
                input_ids, attention_mask = self.tokenize_sequence(sequence)
                labels = torch.full_like(input_ids, -100)
                key_suffix = 'local'

            data[f'input_ids_{key_suffix}'] = input_ids
            data[f'attention_mask_{key_suffix}'] = attention_mask
            data[f'labels_{key_suffix}'] = labels

        return data

    def tokenize_sequence(self, sequence):
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

    def random_mask_sequence(self, sequence):
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoded['input_ids'].squeeze(0)
        labels = torch.full_like(input_ids, -100)

        # Decide where to mask tokens
        mask_indices = (torch.rand(input_ids.shape) < self.mask_probability) & (input_ids != self.tokenizer.pad_token_id)
        labels[mask_indices] = input_ids[mask_indices]

        # 80% time replace with [MASK]
        actual_mask = mask_indices & (torch.rand(input_ids.shape) < 0.8)
        input_ids[actual_mask] = self.tokenizer.mask_token_id

        # 20% time replace with a random token
        random_tokens = torch.randint(2, self.tokenizer.vocab_size, input_ids.shape)
        input_ids[mask_indices & ~actual_mask] = random_tokens[mask_indices & ~actual_mask]

        return input_ids, torch.ones_like(input_ids), labels


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    Collate function to form a batch from multiple data samples.
    Each sample contains tokenized inputs for global and local sequences,
    along with corresponding attention masks and labels if they are part of the batch.
    """
    input_ids_global, attention_mask_global, labels_global = [], [], []
    input_ids_local, attention_mask_local = [], []

    for item in batch:
        if 'input_ids_global_masked' in item:
            input_ids_global.append(item['input_ids_global_masked'])
            attention_mask_global.append(item['attention_mask_global_masked'])
            labels_global.append(item['labels_global_masked'])

        if 'input_ids_local' in item:
            input_ids_local.append(item['input_ids_local'])
            attention_mask_local.append(item['attention_mask_local'])

    # Ensure that the sequence batching includes padding to align all tensors to the same size
    if input_ids_global:
        input_ids_global = pad_sequence(input_ids_global, batch_first=True, padding_value=0)
        attention_mask_global = pad_sequence(attention_mask_global, batch_first=True, padding_value=0)
        labels_global = pad_sequence(labels_global, batch_first=True, padding_value=-100)
    else:
        input_ids_global = torch.Tensor()
        attention_mask_global = torch.Tensor()
        labels_global = torch.Tensor()

    if input_ids_local:
        input_ids_local = pad_sequence(input_ids_local, batch_first=True, padding_value=0)
        attention_mask_local = pad_sequence(attention_mask_local, batch_first=True, padding_value=0)
    else:
        input_ids_local = torch.Tensor()
        attention_mask_local = torch.Tensor()

    return {
        'input_ids_global': input_ids_global,
        'attention_mask_global': attention_mask_global,
        'labels_global': labels_global,
        'input_ids_local': input_ids_local,
        'attention_mask_local': attention_mask_local
    }


#pretraining the bert model


#main class for training:
  1. Process two sets of sequences (global and local) using BERT to extract contextual embeddings.
  2. Integrate these two sets of embeddings using a custom attention mechanism that focuses on relevant parts of the global features for each part of the local features.
  3. Predict an output (like interaction sites or effects) using the combined features.

In [ ]:
import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer

# Define a sequence processor that can handle one channel of input (either global or local)
class SequenceProcessor(nn.Module):
    def __init__(self, model):
        super(SequenceProcessor, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

# Custom attention mechanism to integrate features from both channels
class CustomAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CustomAttention, self).__init__()
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(hidden_size, hidden_size)

    def forward(self, global_features, local_features):
        keys = self.key_layer(global_features)
        queries = self.query_layer(local_features)
        values = self.value_layer(global_features)
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (keys.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)
        context = torch.matmul(attention_weights, values)
        processed_context = self.context_layer(context)
        return processed_context

# Main model to process dual input channels
class ProteinInteractionModel(nn.Module):
    def __init__(self, model_identifier_or_path):
        super(ProteinInteractionModel, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_identifier_or_path)
        hidden_size = self.base_model.config.hidden_size

        self.sequence_processor_global = SequenceProcessor(self.base_model)
        self.sequence_processor_local = SequenceProcessor(self.base_model)
        self.custom_attention = CustomAttention(hidden_size)
        self.mlm_head = nn.Linear(hidden_size, self.base_model.config.vocab_size)

    def forward(self, input_ids_global, attention_mask_global, input_ids_local, attention_mask_local):
        global_features = self.sequence_processor_global(input_ids_global, attention_mask_global)
        local_features = self.sequence_processor_local(input_ids_local, attention_mask_local)
        combined_features = self.custom_attention(global_features, local_features)
        prediction_scores = self.mlm_head(combined_features)
        return prediction_scores


'# Example of initialization and usage\ndef main():\n    model_identifier = \'bert-base-uncased\'  # Or path to your fine-tuned BERT model\n    model = ProteinInteractionModel(model_identifier)\n    model.to(torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\'))\n\n    # Dummy data for testing\n    tokenizer = AutoTokenizer.from_pretrained(model_identifier)\n    text_global = "Protein sequence A"\n    text_local = "Protein sequence B"\n    inputs_global = tokenizer(text_global, return_tensors="pt")\n    inputs_local = tokenizer(text_local, return_tensors="pt")\n\n    # Forward pass\n    outputs = model(inputs_global[\'input_ids\'], inputs_global[\'attention_mask\'],\n                    inputs_local[\'input_ids\'], inputs_local[\'attention_mask\'])\n    print(outputs)\n\nif __name__ == \'__main__\':\n    main()\n'

#Training and Validation data creator:

In [ ]:
#the most important part to check if the class definition and data management is correctly working
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Splitting the dataset into 80% training and 20% testing
train_df, test_df = train_test_split(pairs_df, test_size=0.2, random_state=42)

# Assuming ProteinInteractionDataset is implemented to handle your DataFrame structure
train_dataset = ProteinInteractionDataset(train_df, tokenizer)
test_dataset = ProteinInteractionDataset(test_df, tokenizer)

#since the model isnt runnint we reduced the batch size to half
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


# Model Setup

#Training Loop

In [ ]:
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        # Ensure tensors are not empty and have the correct dimensions
        if batch['input_ids_global'].nelement() == 0 or batch['input_ids_local'].nelement() == 0:
            continue  # Skip the batch if it's empty

        input_ids_global = batch['input_ids_global'].to(device)
        attention_mask_global = batch['attention_mask_global'].to(device)
        labels_global = batch['labels_global'].to(device)
        input_ids_local = batch['input_ids_local'].to(device)
        attention_mask_local = batch['attention_mask_local'].to(device)

        # Check and reshape if necessary to ensure the correct dimensions
        if input_ids_global.dim() == 1:
            input_ids_global = input_ids_global.unsqueeze(0)
        if input_ids_local.dim() == 1:
            input_ids_local = input_ids_local.unsqueeze(0)

        try:
            with autocast():
                prediction_scores = model(input_ids_global, attention_mask_global, input_ids_local, attention_mask_local)
                loss = loss_fn(prediction_scores.view(-1, prediction_scores.size(-1)), labels_global.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        except Exception as e:
            print(f"Failed to process batch: {e}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")


Epoch 1, Avg Loss: 0.0000
Epoch 2, Avg Loss: 0.0000
Epoch 3, Avg Loss: 0.0000
Epoch 4, Avg Loss: 0.0000
